Importing the model from Data folder

In [89]:
from cobra.io import read_sbml_model

In [90]:
model = read_sbml_model('data/iYO844.xml')

Show the model stats

In [91]:
model

Name,iYO844
Memory address,0x01db27a277f0
Number of metabolites,990
Number of reactions,1250
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


Adding surfactin production to the model

In [92]:
from cobra import Reaction, Metabolite

Will add 4 different reactions due to there being 4 different fatty acids that can make up the fatty acid part of surfactin 

In [93]:
new_reaction = Reaction('SP_1') #Surfactin Production reaction 1 

In [94]:
surfactin = Metabolite(id='surfactin_c', compartment='c') 

In [95]:
new_reaction.add_metabolites({model.metabolites.glu__L_c: -1,
                              model.metabolites.leu__L_c: -4,
                              model.metabolites.val__L_c:-1, 
                              model.metabolites.asp__L_c: -1,
                              model.metabolites.nadph_c:-1,
                              model.metabolites.get_by_id('3hddcoa_c'):-1,
                              model.metabolites.atp_c:-1,
                              model.metabolites.adp_c:1,
                              model.metabolites.nadp_c:1,
                              surfactin: 1 
                             })

In [96]:
model.add_reactions([new_reaction])

In [97]:
new_reaction2 = Reaction('SP_2') 

In [98]:
new_reaction2.add_metabolites({model.metabolites.glu__L_c: -1,
                              model.metabolites.leu__L_c: -4,
                              model.metabolites.val__L_c:-1, 
                              model.metabolites.asp__L_c: -1,
                              model.metabolites.nadph_c:-1,
                              model.metabolites.tdcoa_c:-1,
                              model.metabolites.atp_c:-1,
                              model.metabolites.adp_c:1,
                              model.metabolites.nadp_c:1,
                              surfactin: 1 
                             })

In [99]:
model.add_reactions([new_reaction2])

In [100]:
new_reaction3= Reaction('SP_3')

In [101]:
new_reaction3.add_metabolites({model.metabolites.glu__L_c: -1,
                              model.metabolites.leu__L_c: -4,
                              model.metabolites.val__L_c:-1, 
                              model.metabolites.asp__L_c: -1,
                              model.metabolites.nadph_c:-1,
                              model.metabolites.get_by_id('3hhdcoa_c'):-1,
                              model.metabolites.atp_c:-1,
                              model.metabolites.adp_c:1,
                              model.metabolites.nadp_c:1,
                              surfactin: 1 
                             })

In [102]:
model.add_reactions([new_reaction3])

In [103]:
new_reaction4= Reaction('SP_4')

In [104]:
new_reaction4.add_metabolites({model.metabolites.glu__L_c: -1,
                              model.metabolites.leu__L_c: -4,
                              model.metabolites.val__L_c:-1, 
                              model.metabolites.asp__L_c: -1,
                              model.metabolites.nadph_c:-1,
                              model.metabolites.get_by_id('3ohdcoa_c'):-1,
                              model.metabolites.atp_c:-1,
                              model.metabolites.adp_c:1,
                              model.metabolites.nadp_c:1,
                              surfactin: 1 
                             })

In [105]:
model.add_reactions([new_reaction4])

Time to look at production of surfactin

In [106]:
model.add_boundary(model.metabolites.surfactin_c, type='demand') #forcing surfactin production

Reaction identifier,DM_surfactin_c
Name,demand
Memory address,0x01db2866d640
Stoichiometry,surfactin_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [107]:
#flux for reaction 1
with model:
    model.objective = model.reactions.SP_1
    print(model.optimize().objective_value)

0.09298767530811763


The flux for SP_1 is calculated to 0.09

In [108]:
# Remove?
with model:
    model.objective = model.reactions.SP_2
    print(model.optimize().objective_value)

0.08967213114754129


In [109]:
# Remove?
with model:
    model.objective = model.reactions.SP_3
    print(model.optimize().objective_value)

0.08739182532286015


In [110]:
# Remove?
with model:
    model.objective = model.reactions.SP_4
    print(model.optimize().objective_value)

0.08780096308186232


In [111]:
#Flux for surfactin production, so all reactions included 
with model:
    model.objective = model.reactions.DM_surfactin_c
    print(model.optimize().objective_value)

0.09298767530811765


Changing the carbon source to sucrose

In [112]:
model.metabolites.query('suc') #finding sucrose in model - remove?

[<Metabolite argsuc_c at 0x1db27a34310>,
 <Metabolite suc6p_c at 0x1db27ba4c40>,
 <Metabolite sucbz_c at 0x1db27ba4eb0>,
 <Metabolite succ_c at 0x1db27ba4220>,
 <Metabolite succ_e at 0x1db27ba4fa0>,
 <Metabolite succoa_c at 0x1db27ba4ee0>,
 <Metabolite suchms_c at 0x1db27bb8340>,
 <Metabolite sucr_c at 0x1db27bb8460>,
 <Metabolite sucr_e at 0x1db27bb85b0>,
 <Metabolite sucsal_c at 0x1db27bb8790>]

In [113]:
model.metabolites.sucr_e

Metabolite identifier,sucr_e
Name,Sucrose C12H22O11
Memory address,0x01db27bb85b0
Formula,C12H22O11
Compartment,e
In 2 reaction(s),"EX_sucr_e, SUCpts"


In [114]:
model.reactions.EX_glc__D_e.lower_bound = 0

In [115]:
model.reactions.EX_sucr_e.lower_bound = -1000

In [116]:
model.optimize()

,fluxes,reduced_costs
EX_etha_e,0.0,-2.496739e-01
EX_drib_e,0.0,0.000000e+00
EX_csn_e,0.0,-7.490218e-01
EX_5mtr_e,0.0,-2.679884e-17
EX_dtmp_e,0.0,-4.993479e-01
...,...,...
SP_1,0.0,-1.747717e+00
SP_2,0.0,-1.747717e+00
SP_3,0.0,-1.747717e+00
SP_4,0.0,-1.747717e+00


In [117]:
model.reactions.EX_sucr_e.flux

-3.8328393060870605

Calculating the yield

In [ ]:
model.objective = model.reactions.DM_surfactin_c
surfactin_production = model.optimize().objective_value
print('Maximum productivity =', surfactin_production, 'mmol/gDW*h')
   
maximum_yield = surfactin_production / (-1*(model.reactions.EX_sucr_e.flux))
print('Maximum theoretical yield =', maximum_yield, 'mmol-surfactin/mmol-sucrose')

The article had 0.154 mol-surfactin/mol-sucrose after knock-in. 

Sandie starts doing stuff from here:

Compute gene knockout strategies using algorithms like OptKnock, OptGene and OptCouple

In [ ]:
#I did pip install cameo in my own terminal
import cameo

In [ ]:
from cameo.strain_design.heuristic.evolutionary_based import OptGene

In [ ]:
optgene = OptGene(model)

In [ ]:
#Testing if I found the right biomass name
model.reactions.BIOMASS_BS_10

In [ ]:
model.metabolites.DM_surfactin_c

In [ ]:
result = optgene.run(target="DM_surfactin_c",
                     biomass="BIOMASS_BS_10",
                     substrate="EX_sucr_e",
                     max_evaluations=5000,
                     plot=False)

In [ ]:
result = optgene.run(target=model.reactions.psiM, 
                     biomass=model.reactions.DRAIN_Biomass,
                     substrate=model.metabolites.GLCe,
                     max_evaluations=20000, population_size=200, max_knockouts=3,
                     plot=False, growth_coupled=True)